# Hemodynamics and Flow Analysis

Analysis of blood flow dynamics using the Poiseuille flow model and Nernst potential.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, '..')

from quantro_simulator import (
    SimulationConfig, ModelType, OverlayMode,
    create_model, RK4Integrator, NernstModel
)

%matplotlib inline

## 1. Poiseuille Flow Dynamics

In [ ]:
config = SimulationConfig(
    model=ModelType.POISEUILLE,
    overlay_mode=OverlayMode.BASELINE,
    t_start=0.0,
    t_end=20.0,
    dt=0.01
)

model = create_model(config)
time_points, trajectory = RK4Integrator.integrate(model, lambda_param=0.0)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Pressure
axes[0].plot(time_points, trajectory[:, 0], 'r-', linewidth=2)
axes[0].set_xlabel('Time (s)', fontsize=11)
axes[0].set_ylabel('Pressure (mmHg)', fontsize=11)
axes[0].set_title('Arterial Pressure', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Flow rate
axes[1].plot(time_points, trajectory[:, 1], 'b-', linewidth=2)
axes[1].set_xlabel('Time (s)', fontsize=11)
axes[1].set_ylabel('Flow Rate (mL/s)', fontsize=11)
axes[1].set_title('Blood Flow Rate', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Pressure-Flow relationship
axes[2].plot(trajectory[:, 0], trajectory[:, 1], 'g-', linewidth=1.5, alpha=0.7)
axes[2].set_xlabel('Pressure (mmHg)', fontsize=11)
axes[2].set_ylabel('Flow Rate (mL/s)', fontsize=11)
axes[2].set_title('Pressure-Flow Relationship', fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../artifacts/poiseuille_flow.png', dpi=300)
plt.show()

## 2. Vascular Resistance Analysis

In [ ]:
lambda_values = np.linspace(0.0, 1.0, 30)
avg_flow_rates = []
avg_pressures = []

for lam in lambda_values:
    config = SimulationConfig(
        model=ModelType.POISEUILLE,
        overlay_mode=OverlayMode.PARAM_MOD,
        t_end=20.0
    )
    model = create_model(config)
    _, trajectory = RK4Integrator.integrate(model, lam)
    
    avg_pressures.append(np.mean(trajectory[:, 0]))
    avg_flow_rates.append(np.mean(trajectory[:, 1]))

# Calculate effective resistance
resistance = np.array(avg_pressures) / (np.array(avg_flow_rates) + 1e-6)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(lambda_values, avg_flow_rates, 'o-', linewidth=2, markersize=6)
ax1.set_xlabel('Modulation Parameter (λ)', fontsize=11)
ax1.set_ylabel('Average Flow Rate (mL/s)', fontsize=11)
ax1.set_title('Flow Rate vs Parameter', fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(lambda_values, resistance, 's-', color='red', linewidth=2, markersize=6)
ax2.set_xlabel('Modulation Parameter (λ)', fontsize=11)
ax2.set_ylabel('Vascular Resistance', fontsize=11)
ax2.set_title('Effective Resistance vs Parameter', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../artifacts/vascular_resistance.png', dpi=300)
plt.show()

## 3. Nernst Potential and Ion Dynamics

In [ ]:
config = SimulationConfig(
    model=ModelType.NERNST,
    overlay_mode=OverlayMode.BASELINE,
    t_end=50.0
)

model = create_model(config)
time_points, trajectory = RK4Integrator.integrate(model, lambda_param=0.2)

# Calculate Nernst potentials
potentials = [model.calculate_potential(state) for state in trajectory]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(time_points, trajectory[:, 0], 'b-', linewidth=2, label='Intracellular')
axes[0, 0].plot(time_points, trajectory[:, 1], 'r-', linewidth=2, label='Extracellular')
axes[0, 0].set_xlabel('Time', fontsize=11)
axes[0, 0].set_ylabel('Concentration (mM)', fontsize=11)
axes[0, 0].set_title('Ion Concentrations', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(time_points, potentials, 'g-', linewidth=2)
axes[0, 1].set_xlabel('Time', fontsize=11)
axes[0, 1].set_ylabel('Nernst Potential (mV)', fontsize=11)
axes[0, 1].set_title('Membrane Potential', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(trajectory[:, 0], trajectory[:, 1], 'purple', linewidth=2)
axes[1, 0].set_xlabel('Intracellular [Ion] (mM)', fontsize=11)
axes[1, 0].set_ylabel('Extracellular [Ion] (mM)', fontsize=11)
axes[1, 0].set_title('Concentration Phase Space', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

gradient_ratio = trajectory[:, 1] / (trajectory[:, 0] + 1e-6)
axes[1, 1].plot(time_points, gradient_ratio, 'orange', linewidth=2)
axes[1, 1].set_xlabel('Time', fontsize=11)
axes[1, 1].set_ylabel('Concentration Ratio (out/in)', fontsize=11)
axes[1, 1].set_title('Concentration Gradient', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../artifacts/nernst_dynamics.png', dpi=300)
plt.show()

print(f"Final Nernst potential: {potentials[-1]:.2f} mV")

## Summary

Hemodynamic insights:
- Poiseuille flow captures pressure-flow relationships
- Vascular resistance modulated by parameter changes
- Nernst potential tracks ion concentration gradients

**Clinical Applications**:
- Hypertension modeling
- Electrolyte imbalance detection
- Cardiac output estimation